In [ ]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from datetime import datetime,timedelta

def GetTable(date):

    Date = str(StartDate).replace("-",".").replace(" ",".")[:-6]
    url = "https://www.weather.go.kr/weather/observation/currentweather.jsp?auto_man=m&stn=0&type=t11&reg=100&tm="+Date+"%3A00&x=16&y=5"   
    res = requests.get(url)
    soup = BeautifulSoup(res.text,"html.parser")
    table = soup.select('table') 
    table = str(table[0])
    table_df_list = pd.read_html(table)
    
    return [table_df_list[0], soup]

# Range of days you want to see
StartDate = datetime(year,month,day,14,0) # Never mind with numbers 14 and 0. Just put year, month, and day in.
EndDate = datetime(2010,1,3,14,0)

# Range of hours that you want to see everyday
StartHour = 8
EndHour = 18

DateRange = (EndDate - StartDate).days
HourRange = EndHour - StartHour

# Make the Resultant diagram to fill in 
table_df= GetTable(StartDate)[0]
Result = table_df.iloc[:,:2]
Result.rename(columns = {'01H' : 'SubZeroRate'}, inplace = True)
Result["SubZeroRate"] = 0

# Fill the diagram through surfing the website
time = StartDate
while((time - EndDate).days < 1):
    [table_df, soup] = GetTable(time)

    # Shows which date the program is digging in 
    date = soup.select_one('#observation_text')
    date = date["value"]
    print(date)

    # Defines the specific range of hours to take into account
    Worktime = pd.concat([table_df.iloc[:,0],table_df.iloc[:,StartHour:EndHour+1]],axis = 1)

    # Filter out the hours which are warmer than zero deg.
    Cold = Worktime[Worktime.iloc[:,1:]<0]
    Cold = np.array(Cold)

    # Calculate the overall subzero hours within each region
    for i in range(len(Cold)):
        num = (Cold[i]<0).sum()
        Result.iloc[i,1] += num
        
    time += timedelta(days = 1)
        
for i in range(len(Result)):
    Result.iloc[i,1] = Result.iloc[i,1]/(DateRange * HourRange)
    
print(Result)
# Stores the diagram as an excel file.
Result.to_excel('path/that/you/want/Korea_Static_Crawling.xlsx')